In [1]:
from scripts import utils

import os
from pprint import pprint
import openai

openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]), "\n")
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'> 

Doc ID: e1e2d226-c596-417a-bb96-0769693eaeff
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [6]:
documents[1].text

'PAGE 2"AI is the new \nelectricity. It will \ntransform and improve \nall areas of human life."\nAndrew Ng'

### Basic RAG Pipeline

In [3]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [8]:
# print(document.text)

In [4]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex.from_documents([document], service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
query_engine = index.as_query_engine()

In [7]:
pprint(query_engine.query(
    "Who is Bola Ahmed Tinubu?"
))

Response(response="I'm sorry, but I don't have any information about Bola "
                  'Ahmed Tinubu based on the given context.',
         source_nodes=[NodeWithScore(node=TextNode(id_='dc1623d5-a225-485a-824c-db0aae469480', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3628187d-abda-4457-bb01-d2db09c41aba', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='fdc7ce9a017ce62d4e0b30cc419051a9bb56618a8569f51a2b64f8535fe6244e'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='078611d9-e01a-49a3-9954-0b4e451a93d6', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='26acbe7ab4f830acb42e62964fb13c92f3081ba58b41f43231766d1b68c6f3f2')}, hash='ecc59f2243975343f9724f9707be820b816049cea8cab8eb04a184a6fff08fa1', text='PAGE 37Overcoming Imposter \nSyndromeCHAPTER 11\n\nPAGE 38Before we dive into the final chapter of this book, I’d like to address the serio

In [11]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
pprint(str(response))

('When finding projects to build your experience, there are several steps you '
 'can take. First, you can join existing projects by asking to join someone '
 "else's project if they have an idea. Additionally, you can continue reading, "
 'taking courses, and talking to domain experts to come up with new ideas. '
 'Another approach is to focus on a specific application area where machine '
 'learning has not yet been applied. This can give you the opportunity to '
 'explore unique and creative applications that no one else has done yet. '
 'Finally, you can develop a side hustle or personal project that may or may '
 'not develop into something bigger, as this can stir your creative juices and '
 'strengthen bonds with collaborators.')


### Evaluation setup using TruLens

In [12]:
eval_questions = []
with open("texts/eval_questions.txt", "r") as file:
    for line in file:
        # Remove newline character 
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [13]:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [14]:
pprint(eval_questions)

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter '
 'syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'What is the right AI job for me?']


In [1]:
from trulens_eval import Tru

tru = Tru(database_url="sqlite:///db/default.sqlite")

# tru.reset_database()

🦑 Tru initialized with db url sqlite:///db/default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [37]:
from scripts.utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id="Direct Query Engine")

In [38]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [30]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [31]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_6aa96a48bb6ce0424fc6a1270f807f3c,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_6aa96a48bb6ce0424fc...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T16:39:20.174730"", ""...",2024-01-10T16:39:31.861304,0.9,0.6,1.00,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",11,2182,0.003355
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9329c6256dfb24b2f166f167f3b4f309,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_9329c6256dfb24b2f16...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T16:39:33.174192"", ""...",2024-01-10T16:39:41.607836,0.9,0.0,1.00,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,8,1722,0.002632
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_aefe240795d018984848e482006083c7,"""What is the importance of networking in AI?""","""Networking is important in AI because it help...",-,"{""record_id"": ""record_hash_aefe240795d01898484...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T16:39:42.995205"", ""...",2024-01-10T16:39:51.533584,0.9,0.1,0.25,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,8,1713,0.002613
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fa33f5e3141d34fe7b7d07ad0e95198f,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_fa33f5e3141d34fe7b7...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T16:39:53.129896"", ""...",2024-01-10T16:40:01.787373,0.9,0.6,0.82,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,8,1694,0.002590
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_ed0ff1fb814540f7502446009d91b031,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_ed0ff1fb814540f7502...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T16:40:03.466133"", ""...",2024-01-10T16:40:12.487700,0.9,0.3,0.40,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,9,1710,0.002623


In [2]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.161:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Advanced RAG Pipeline

#### Sentence Window Retrieval

In [21]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [22]:
from scripts.utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
)

In [23]:
from scripts.utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [25]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
pprint(str(window_response))

('To get started on a personal project in AI, it is important to first '
 'identify and scope the project. Consider your career goals and choose a '
 'project that complements them. Ensure that the project is responsible, '
 'ethical, and beneficial to people. As you progress in your career, aim for '
 'projects that grow in scope, complexity, and impact over time. Building a '
 'portfolio of projects that shows skill progression can also be helpful. '
 'Additionally, there are resources available in the book that provide '
 'guidance on starting your AI job search and finding the right AI job for '
 'you.')


In [33]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine, app_id="Sentence Window Query Engine"
)

In [34]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4dd05ca010 is calling an instrumented method <function BaseQueryEngine.query at 0x7f4e14561120>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4dd05ca010 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x7f4dd05cbd10 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f4e04bd0ea0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obj

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4dd05ca010 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4dd0574110 is calling an instrumented method <function Refine.get_response at 0x7f4e04b9ea20>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.


How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively as a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions, and steer the project in the right direction. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to improved project outcomes.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4dd05ca010 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4dd0574110 is calling an instrumented method <function Refine.get_response at 0x7f4e04b9ea20>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include keeping up-to-date with evolving technology, finding suitable projects and estimating timelines and return on investment, managing the highly iterative nature of AI projects, collaborating with stakeholders who lack expertise in AI, and struggling with technical challenges while reading research papers or tuning neural network hyperparameters.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4dd05ca010 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4dd0574110 is calling an instrumented method <function Refine.get_response at 0x7f4e04b9ea20>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.


Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. The author of the text acknowledges that they still find many research papers challenging to read and have made mistakes while working with neural networks. They assure the reader that everyone who has published a seminal AI paper has also faced similar technical challenges at some point.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4dd05ca010 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4dd0574110 is calling an instrumented method <function Refine.get_response at 0x7f4e04b9ea20>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.


What is the right AI job for me?
The right AI job for you is one that complements your career goals and allows you to showcase skill progression through a portfolio of projects. It is important to start your AI job search with a simple framework and utilize informational interviews to find the right job. Building a career in AI requires overcoming imposter syndrome and understanding the keys to success in the field.


In [39]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.400000,0.872727,0.843182,10.909091,0.000839
Direct Query Engine,0.269231,0.915385,0.739744,10.615385,0.003047


#### Auto Merging Retrieval

In [40]:
from scripts.utils import build_automerging_index

automerging_index = build_automerging_index(
    documents, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="merging_index"
)

In [41]:
from scripts.utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [42]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 52e4d77d-1208-4fed-933b-b8a878e747de.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 72c07f49-c944-433d-a812-0e7fe3eb9686.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones. This progression over time will demonstrate your growth and development in the field. Additionally, effective communication is crucial in order to explain your thinking and showcase the value of your work. Being able to articulate your ideas will help others see the potential in your projects and trust you with the necessary resources.


In [43]:
tru_recorder_automerging = get_prebuilt_trulens_recorder(
    automerging_query_engine, app_id="Automerging Query Engine"
)

In [44]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4db2e10290 is calling an instrumented method <function BaseQueryEngine.query at 0x7f4e14561120>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4db2e10290 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f4db80c14d0 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f4e04bd0ea0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: e2322408-3f31-4a3f-9642-4fd2fa94e2af.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 3cf60441-7d20-4786-931d-a7ddc4e89fa3.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4db8235510 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f4e04b9d760>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4db8235510 is calling an instrumented method <function Refine.get_response at 0x7f4e04b9ea20>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.


What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job. These steps are supported by being part of a community.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4db2e10290 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4db8235510 is calling an instrumented method <function Refine.get_response at 0x7f4e04b9ea20>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.


...

Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. The context information suggests that even accomplished individuals in the AI community have faced technical challenges and struggled with certain aspects of AI. The author encourages newcomers to not be discouraged by these challenges and assures them that everyone has experienced similar difficulties at some point.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f4db2e10290 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f4e0199b740>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f4e002c9d10) using this function.


> Merging 1 nodes into parent node.
> Parent node id: 69f9d5c8-7ff5-4278-9709-bd598f99742b.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 2c8277d4-e7d5-4e0e-8c5a-dc03115573a9.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 13645683-8f23-48d7-a915-fd601775d1d9.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f4db8235510 is calling an instrumented method <function Refine.get_response at 0x7f4e04b9ea20>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f4ddd912350) using this function.


What is the right AI job for me?
The right AI job for you is discussed in Chapter 9 of the eBook "How to Build a Career in AI."


In [45]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.540909,0.827273,0.894697,10.909091,0.000851
Sentence Window Query Engine,0.400000,0.872727,0.843182,10.909091,0.000839
Direct Query Engine,0.269231,0.915385,0.739744,10.615385,0.003047
